In [2]:
from PIL import Image

# Load the image
image = Image.open("/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/fake_train/image_137.png")

# Get dimensions
width, height = image.size

print(f"Width: {width}, Height: {height}")

Width: 128, Height: 128


In [3]:
train_dataset_path = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
#     validation_split=0.2,  # Split the data into train/validation sets
)

# Set up the training data generator with augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1./255, 
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     brightness_range=[0.8, 1.2], 
#     shear_range=0.2, 
#     channel_shift_range=0.1,
#     validation_split=0.2  # Split the data for validation
# )

IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 32

# Create training generator
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=42,
    subset='training'  # Specify this generator as training
)

# Create validation generator
# validation_generator = train_datagen.flow_from_directory(
#     train_dataset_path,
#     target_size=(IMG_WIDTH, IMG_HEIGHT),
#     batch_size=BATCH_SIZE,
#     class_mode='binary',
#     shuffle=True,
#     seed=42,
#     subset='validation'  # Specify this generator as validation
# )

Found 1600 images belonging to 2 classes.


In [5]:
print(train_generator.class_indices)

{'fake_train': 0, 'real_train': 1}


In [6]:
import os

# Path to the folder containing the photos
folder_path = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/fake_train'  # Replace with your folder path

# Function to count images in the specified folder
def count_images(folder_path):
    # Count images with specific extensions
    return len([file for file in os.listdir(folder_path) if file.endswith(('.png', '.jpg', '.jpeg', '.gif'))])

# Count images in the folder
num_images = count_images(folder_path)

# Print the result
print(f'Number of photos in the folder: {num_images}')

Number of photos in the folder: 800


In [7]:
import os

# Path to the folder containing the photos
folder_path = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/real_train'  # Replace with your folder path

# Function to count images in the specified folder
def count_images(folder_path):
    # Count images with specific extensions
    return len([file for file in os.listdir(folder_path) if file.endswith(('.png', '.jpg', '.jpeg', '.gif'))])

# Count images in the folder
num_images = count_images(folder_path)

# Print the result
print(f'Number of photos in the folder: {num_images}')

Number of photos in the folder: 800


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from sklearn.utils import class_weight

from tensorflow.keras.preprocessing.image import ImageDataGenerator

labels = train_generator.classes

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(labels),
                                                  y=labels)
class_weights_dict = dict(enumerate(class_weights))

# Create a CNN model
model = Sequential()

# Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(128,128, 3)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block 5
# model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))


# Classification block
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
import tensorflow as tf
model.compile('adam',loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,194,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,715,073 (17.99 MB)

 Trainable params: 4,715,073 (17.99 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(train_generator,
                    epochs=50,batch_size=32,
#                     validation_data=validation_generator,
                    class_weight=class_weights_dict)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727983178.509384     106 service.cc:145] XLA service 0x7fe904004490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727983178.509455     106 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727983178.509460     106 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 3/50 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.5365 - loss: 0.7407

I0000 00:00:1727983183.089506     106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 127ms/step - accuracy: 0.5092 - loss: 0.7062
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.5376 - loss: 0.6790
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.4880 - loss: 0.6947
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.4773 - loss: 0.6944
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.5883 - loss: 0.6766
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.6574 - loss: 0.6198
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.7853 - loss: 0.4622
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.8749 - loss: 0.2836
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.9086 - loss: 0.2245
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.9358 - loss: 0.1624
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.9560 - loss: 0.1314
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.9640 -

In [12]:
history = model.fit(train_generator,
                    epochs=50,batch_size=32,
#                     validation_data=validation_generator,
                    class_weight=class_weights_dict)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 7.3226e-06
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 8.3592e-06
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 1.0000 - loss: 4.9503e-06
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 1.0000 - loss: 4.6599e-06
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 3.4164e-06
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 2.9717e-06
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 2.5867e-06
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 1.0000 - loss: 6.5699e-06
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 3.9755e-06
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 1.0000 - loss: 2.9800e-06
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 1.0000 - loss: 1.4390e-06
Epoch 12/50
50/50 ━

In [14]:
history = model.fit(train_generator,
                    epochs=50,batch_size=32,
#                     validation_data=validation_generator,
                    class_weight=class_weights_dict)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 2.8268e-07
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 2.6493e-07
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 2.9307e-07
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 1.2507e-07
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 1.0000 - loss: 1.5934e-07
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 1.8081e-07
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 1.0000 - loss: 2.0340e-07
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 2.7927e-07
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 1.0000 - loss: 1.1978e-07
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 1.0000 - loss: 1.4900e-07
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 1.0000 - loss: 1.2998e-07
Epoch 12/50
50/50 ━

In [15]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load your trained model (make sure to specify the correct path)
# model = load_model('path_to_your_trained_model.h5')  # Update with your model path

# Function to preprocess the image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Match the size used during training
    img_array = image.img_to_array(img)  # Convert the image to a NumPy array
    img_array = img_array / 255.0  # Rescale pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Path to the test images directory
test_images_dir = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/test_images/test_images'  # Update with the correct path

# Create a list to hold the predictions
predictions = []

# Iterate through all the images in the directory
for img_filename in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, img_filename)
    
    # Ensure it's a file and has the correct extension
    if os.path.isfile(img_path) and img_filename.endswith('.jpg'):
        # Load and preprocess the image
        processed_image = load_and_preprocess_image(img_path)
        
        # Make a prediction
        prediction = model.predict(processed_image)
        
        # Interpret the result
        predicted_label = 'fake' if prediction[0][0] < 0.5 else 'real'
        
        # Append the result to the predictions list
        predictions.append({'filename': img_filename, 'label': predicted_label})

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions)

# Save to CSV file
predictions_df.to_csv('image_predictions12.csv', index=False)

print('Predictions saved to image_predictions12.csv')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

In [62]:
# Load and preprocess a PNG image
image_path = '/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/train_images/train_images/fake_train/image_1000.png'  # Replace with your PNG image path
processed_image = load_and_preprocess_image(image_path)

# Make a prediction
prediction = model.predict(processed_image)

# Interpret the result
predicted_label = 'real' if prediction[0][0] < 0.5 else 'fake'  # Adjust threshold if necessary

# Print the predicted label
print(f'The predicted label is: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
The predicted label is: real


In [ ]:
#first i did this using validation dataset, and it gave more somewhere around 98% accuracy on validation, so the model is not overfitting
#then i removed the validation to get more images to get trained in case if it givves me better f1 score and it did